In [ ]:
"""
This notebook provides the code necessary to fully register one of the datasets analyzed in the Denditome MORF project.

Please download this notebook and place it into an empty directory.
- All of the necessary GitHub repositiories will be cloned to the current working directory
- All of the necessary non-native Python libraries will be installed to the current working directory
- All of the necessary data will be downloaded + unzipped to the current working directory
- Several folders to save the outputs from this registration pipeline will be created and filled during the registration process

Note: There will be several RuntimeWarning and UserWarning messages during the registration process. This is the expected behavior and these messages can be disregarded.
Note: This script has only been tested on a Linux system, and therefore may not on a system operating on Windows OS or Mac OS.
"""

In [ ]:
import os
import sys
import requests

sys.path.append(os.getcwd())

## Clone necessary code repositories from GitHub, install non-native Python libraries, and download example dataset + atlases

In [ ]:
# Clone necessary repos from GitHub
os.mkdir("registration_docs")
os.system("git clone https://github.com/twardlab/registration_docs registration_docs")

os.mkdir("emlddmm")
os.system("git clone https://github.com/twardlab/emlddmm emlddmm")

os.mkdir("donglab_workflows")
os.system("git clone https://github.com/twardlab/donglab_workflows donglab_workflows")

# Install necessary non-native Python libraries in order to run the 3 scripts
os.system("pip install argparse numpy matplotlib nibabel scipy scikit-image")
os.system("pip3 install torch torchvision torchaudio")

# Download example dataset + atlases from twardlab.com
data_url = 'https://twardlab.com/data/morf/dendritome_data.zip'
response = requests.get(data_url, stream=True)

if response.status_code == 200:
    with open('dendritome_data.zip', 'wb') as f:
        for chunk in response.iter_content(1024):
            f.write(chunk)
    print('Zip file downloaded successfully')
else:
    print('Failed to download zip file. Please check that your network connection is stable.')

# os.system("sudo -S apt install unzip")
os.system("unzip -d dendritome_data dendritome_data.zip")

## Step 1 (Atlas to low resolution)

The below cell will register the Allen Mouse Brain Atlas to all of the low resolution images. The outputs from this cell will be saved in 'step1_outputs' and used in Step 3.

In [ ]:
script_path = 'registration_docs/source/step1_atlas_registration.py'

# Example where all positional/required arguments, to_flip, device, and A are provided
brain = 'TME08-1'
orientation = 'W'
target_files =  ['dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_01_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_02_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_03_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_04_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_05_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_06_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_07_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_08_ch_0_pow_[0.125]_down.npz']

outdir = 'step1_outputs'
if not os.path.exists(outdir):
    os.mkdir(outdir)

atlas_names = ['dendritome_data/atlases/ara_nissl_50.vtk',
               'dendritome_data/atlases/average_template_50.vtk',
               'dendritome_data/atlases/annotation_50.vtk']

to_flip = [0,1,3,5,7]
n_iter = 5000 # Note that in practice, this variable is normally set to 40000. In this example, we decided to decrease it to 5000 in order to save time for the user.
device = 'cuda:0'
d_path = 'donglab_workflows'
e_path = 'emlddmm'
A = "[[-1.0,0.0,0.0,500.0],[0.0,0.0,-1.0,0.0],[0.0,1.0,0.0,0.0],[0.0,0.0,0.0,1.0]]" 

cmd_str = f'python3 {script_path} {brain} {orientation} -low_paths {target_files[0]} {target_files[1]} {target_files[2]} {target_files[3]} {target_files[4]} {target_files[5]} {target_files[6]} {target_files[7]} -outdir {outdir} -atlas_paths {atlas_names[0]} {atlas_names[1]} {atlas_names[2]} -to_flip {to_flip[0]} {to_flip[1]} {to_flip[2]} {to_flip[3]} {to_flip[4]} -n_iter {n_iter} -device {device} -d_path {d_path} -e_path {e_path} -A {A}'

# Optional flags to include
save_fig0 = False
save_fig1 = False
save_fig2 = False
save_fig3 = False
save_fig4 = False
save_fig5 = False
save_fig6 = False
normalizeData = True
zeroMean = True
largerRGBfig = False

if save_fig0:
    cmd_str += ' -save_fig0'
if save_fig1:
    cmd_str += ' -save_fig1'
if save_fig2:
    cmd_str += ' -save_fig2'
if save_fig3:
    cmd_str += ' -save_fig3'
if save_fig4:
    cmd_str += ' -save_fig4'
if save_fig5:
    cmd_str += ' -save_fig5'
if save_fig6:
    cmd_str += ' -save_fig6'

if not normalizeData:
    cmd_str += ' -norm'
if not zeroMean:
    cmd_str += ' -zm'
if largerRGBfig:
    cmd_str += ' --largerRGBfig'

print('Starting registration of atlas data to 10x data...')
os.system(cmd_str)
print('Finished registration of atlas data to 10x data')

## Step 2 Example (High resolution to low resolution)

The below cell will register all of the high resolution images to all of the low resolution images. In particular, it will iterate over each high resolution image and register this image to the corresponding low resolution image. The outputs from this cell will be saved in 'step2_outputs' and used in Step 3

In [ ]:
script_path = 'registration_docs/source/step2_atlas_registration.py'
dataset = 'TME08-1'

low_img_paths = ['dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_01_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_02_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_03_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_04_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_05_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_06_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_07_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_08_ch_0_pow_[0.125]_down.npz']

high_img_paths = ['dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_01A_ch_0_pow_[0.125]_down.npz', 
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_01B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_02A_ch_0_pow_[0.125]_down.npz', 
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_02B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_03A_ch_0_pow_[0.125]_down.npz', 
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_03B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_04A_ch_0_pow_[0.125]_down.npz', 
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_04B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_05A_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_05B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_06A_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_06B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_07A_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_07B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_08A_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_08B_ch_0_pow_[0.125]_down.npz']

all_high_num = ['01A', '01B','02A', '02B', '03A', '03B', '04A', '04B', '05A', '05B', '06A', '06B', '07A', '07B', '08A', '08B']

outdir = 'step2_outputs'
if not os.path.exists(outdir):
    os.mkdir(outdir)

# In practice, these shifts need to be manually determined. This can be done by setting checkInit to True and iteratively modifying the shifts 
# until the low and high resolution images have a reasonable initial alignment. 
# The following shifts were manually determined by a researcher who registered this specific dataset.
all_shifts = [[0,1000,2000], [0,1000,-1100], [0,500,2000], [0,600,-1800], [0,500,2700], [0,500,-1900], [0,800,2900], [0,1000,-2000], [0,300,3200], [0,100,-2700], [0,500,3300], [0,300,-3200], [0,300,3500], [0,200,-3300], [0,500,3200], [0,300,-3800]]

e_path = 'emlddmm'
d_path = 'donglab_workflows'

gamma = False
checkInit = False
zeroMean = True
useRigidTransform = False

low_idx = 0
# Register every high_res image
for high_idx in range(len(high_img_paths)):

    # Increment the low_idx every other 
    if high_idx != 0 and high_idx % 2 == 0:
        low_idx += 1

    # Define the arguments
    low_path = low_img_paths[low_idx]
    high_path = high_img_paths[high_idx]
    high_num = all_high_num[high_idx]
    shifts = all_shifts[high_idx]

    cmd_str = f'python3 {script_path} {dataset} -low_path {low_path} -high_path {high_path} -high_num {high_num} -outdir {outdir} -shifts {shifts[0]} {shifts[1]} {shifts[2]} -e_path {e_path} -d_path {d_path}'

    # Append flags if necessary
    if gamma:
        cmd_str += ' -gamma'
    
    if checkInit:
        cmd_str += ' -checkInit'
    
    if not zeroMean:
        cmd_str += ' -zeromean'
    
    if useRigidTransform:
        cmd_str += ' -useRigidTransform'
    
    # Run the command line script as defined above
    print(f'Starting registration of the high_res img ({high_path}) to the low_res img ({low_path})...')
    os.system(cmd_str)
    print(f'Finished registration of the high_res img ({high_path}) to the low_res img ({low_path})')

## Step 3 Example

The below cell will register all of the neuron reconstruction (.swc) files to the high resolution images. The outputs of this cell will be saved in 'step3_outputs' and will contain the combined low_res/high_res/neuron data for every low_res slice, and the registered .swc files for every neuron. 

In [ ]:
script_path = 'registration_docs/source/step3_atlas_registration.py'

# Define all input variables
dataset = 'TME08-1'
orientation = 'W'
neuron_dir = 'dendritome_data/neuron_reconstructions'
low_to_high_dir = 'step2_outputs'
tform = 'step1_outputs/transformation_outputs.npz'

low_img_paths = ['dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_01_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_02_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_03_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_04_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_05_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_06_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_07_ch_0_pow_[0.125]_down.npz',
                 'dendritome_data/10x_images/Camk2a-MORF3-D1Tom_TME08-1_10x_08_ch_0_pow_[0.125]_down.npz']

low_imd_ids = ['01','02','03','04','05','06','07','08']

high_img_paths = ['dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_01A_ch_0_pow_[0.125]_down.npz', 
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_01B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_02A_ch_0_pow_[0.125]_down.npz', 
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_02B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_03A_ch_0_pow_[0.125]_down.npz', 
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_03B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_04A_ch_0_pow_[0.125]_down.npz', 
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_04B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_05A_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_05B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_06A_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_06B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_07A_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_07B_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_08A_ch_0_pow_[0.125]_down.npz',
                  'dendritome_data/30x_images/Camk2a-MORF3-D1Tom_TME08-1_30x_Str_08B_ch_0_pow_[0.125]_down.npz']

outdir = 'step3_outputs'
if not os.path.exists(outdir):
    os.mkdir(outdir)

atlas_paths = ['dendritome_data/atlases/ara_nissl_50.vtk',
               'dendritome_data/atlases/average_template_50.vtk',
               'dendritome_data/atlases/UPenn_labels_reoriented_origin.vtk',
               'dendritome_data/atlases/atlas_info_KimRef_FPbasedLabel_v2.7.csv']

e_path = 'emlddmm'
d_path = 'donglab_workflows'

cmd_str = f'python3 {script_path} {dataset} {orientation} -neuron_dir {neuron_dir} -low_to_high_dir {low_to_high_dir} -tform {tform} -low_img_paths {low_img_paths[0]} {low_img_paths[1]} {low_img_paths[2]} {low_img_paths[3]} {low_img_paths[4]} {low_img_paths[5]} {low_img_paths[6]} {low_img_paths[7]} -low_img_ids {low_imd_ids[0]} {low_imd_ids[1]} {low_imd_ids[2]} {low_imd_ids[3]} {low_imd_ids[4]} {low_imd_ids[5]} {low_imd_ids[6]} {low_imd_ids[7]} -high_img_paths {high_img_paths[0]} {high_img_paths[1]} {high_img_paths[2]} {high_img_paths[3]} {high_img_paths[4]} {high_img_paths[5]} {high_img_paths[6]} {high_img_paths[7]} {high_img_paths[8]} {high_img_paths[9]} {high_img_paths[10]} {high_img_paths[11]} {high_img_paths[12]} {high_img_paths[13]} {high_img_paths[14]} {high_img_paths[15]} -outdir {outdir} -atlas_paths {atlas_paths[0]} {atlas_paths[1]} {atlas_paths[2]} {atlas_paths[3]} -e_path {e_path} -d_path {d_path}'

toggle_seg = True
toggle_cp = True
toggle_low = True
toggle_high = True
toggle_neurons = True

if not toggle_seg:
    cmd_str += ' -toggle_seg'
if not toggle_cp:
    cmd_str += ' -toggle_cp'
if not toggle_low:
    cmd_str += ' -toggle_low'
if not toggle_high:
    cmd_str += ' -toggle_high'
if not toggle_neurons:
    cmd_str += ' -toggle_neurons'

print('Starting registration of swc data with all of the low res data...')
os.system(cmd_str)
print('Finished registration of swc data with all of the low res data')